## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [ ]:
import random
N = 1_000_000
A = [random.randint(0,1000) for i in range (N)]
B = [i + 100 for i in A]
average = sum(B)/N
average

600.640027

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [ ]:
import pandas as pd
import numpy as np
import random
import string

# Создание таблицы с 2млн строк и 4 столбцами со случайными числами
data = pd.DataFrame(np.random.randint(0, 1000, size=(2000000, 4)), columns=list('ABCD'))

# Добавление столбца 'key' со случайными английскими буквами
def random_string(length):
    """Функция генерации случайной строки из английских букв"""
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

data['key'] = [random_string(5).upper() for _ in range(len(data))]

# Выбор подмножества строк, где в стобце 'key' указаны первые 5 английских букв
subset = data[data['key'].str.contains('^[A-Z]{5}')]

# Вывод первых 10 строк из выбранного подмножества
print(subset.head(10))


     A    B    C    D    key
0  256  130   27  734  LOTQT
1  225  489  209  981  NACML
2  934  779  170  667  KAMXL
3  129  206  679  218  BMLKS
4  572  756  415  658  RWFRF
5  264  732  793  814  FSVOM
6  489  155  448   50  IZBIO
7   11  270  387  266  ZLWHN
8  558  302  133  183  LOLLI
9  189  737  466  572  TYGBE


## Лабораторная работа 3

In [ ]:
# !pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [7]:
import pandas as pd
import time
recipes = pd.read_csv('recipes_sample.csv')
reviews = pd.read_csv('reviews_sample.csv')

# Приведение столбцов к нужным типам
recipes['id'] = recipes['id'].astype(int)
recipes['minutes'] = recipes['minutes'].astype(int)

reviews['user_id'] = reviews['user_id'].astype(int)
reviews['recipe_id'] = reviews['recipe_id'].astype(int)
reviews['rating'] = reviews['rating'].astype(float)
reviews['date'] = pd.to_datetime(reviews['date'])

def averge_value(file):
    summ = 0
    count = 0
    for i, row in file.iterrows():
        if row['date'].year == 2010:
            summ += row['rating']
            count += 1
    return summ/count

def averge_value2(file):
    new_file = file[file['date'].dt.year == 2010]
    summ = 0
    count = 0
    for i, row in new_file.iterrows():
        summ += row['rating']
        count += 1
    return summ/count

def averge_value3(file):
    new_file = file[file['date'].dt.year == 2010]
    return new_file['rating'].mean()

start_time = time.time()
av_val = averge_value(reviews)
end_time = time.time()
print(f"Средний рейтинг А: {av_val} Время выполнения: {end_time-start_time}")

start_time = time.time()
av_val2 = averge_value2(reviews)
end_time = time.time()
print(f"Средний рейтинг В: {av_val2} Время выполнения: {end_time-start_time}")

start_time = time.time()
av_val3 = averge_value3(reviews)
end_time = time.time()
print(f"Средний рейтинг С: {av_val3} Время выполнения: {end_time-start_time}")  

Средний рейтинг А: 4.4544402182900615 Время выполнения: 6.42448616027832
Средний рейтинг В: 4.4544402182900615 Время выполнения: 0.5293030738830566
Средний рейтинг С: 4.4544402182900615 Время выполнения: 0.008047103881835938


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [9]:
%load_ext line_profiler

%lprun -f averge_value averge_value(reviews)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


    Timer unit: 1e-07 s
    Total time: 27.0202 s
    
Function: averge_value at line 15

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    15                                           def averge_value(file):
    16         1         24.0     24.0      0.0      summ = 0
    17         1         14.0     14.0      0.0      count = 0
    18    126696  235847119.0   1861.5     87.3      for i, row in file.iterrows():
    19    114602   31463876.0    274.5     11.6          if row['date'].year == 2010:
    20     12094    2797284.0    231.3      1.0              summ += row['rating']
    21     12094      94017.0      7.8      0.0              count += 1
    22         1          7.0      7.0      0.0      return summ/count

In [10]:
%lprun -f averge_value2 averge_value2(reviews)

    Timer unit: 1e-07 s
    Total time: 2.75646 s

Function: averge_value2 at line 24

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    24                                           def averge_value2(file):
    25         1     199394.0 199394.0      0.7      new_file = file[file['date'].dt.year == 2010]
    26         1          7.0      7.0      0.0      summ = 0
    27         1          7.0      7.0      0.0      count = 0
    28     12094   23766224.0   1965.1     86.2      for i, row in new_file.iterrows():
    29     12094    3517101.0    290.8     12.8          summ += row['rating']
    30     12094      81848.0      6.8      0.3          count += 1
    31         1          8.0      8.0      0.0      return summ/count

In [11]:
%lprun -f averge_value3 averge_value3(reviews)

    Timer unit: 1e-07 s
    Total time: 0.0298065 s
    
Function: averge_value3 at line 33

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    33                                           def averge_value3(file):
    34         1     286082.0 286082.0     96.0    new_file = file[file['date'].dt.year == 2010]
    35         1      11983.0  11983.0      4.0    return new_file['rating'].mean()

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [12]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [13]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

    Timer unit: 1e-07 s
    Total time: 99.6916 s

Function: get_word_reviews_count at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_word_reviews_count(df):
     2         1         15.0     15.0      0.0      word_reviews = {}
     3    126679  240801682.0   1900.9     24.2      for _, row in df.dropna(subset=['review']).iterrows():
     4    126679   60736804.0    479.5      6.1          recipe_id, review = row['recipe_id'], row['review']
     5    126679    7369321.0     58.2      0.7          words = review.split(' ')
     6   6792010   31060018.0      4.6      3.1          for word in words:
     7   6617066   50879518.0      7.7      5.1              if word not in word_reviews:
     8    174944    1455579.0      8.3      0.1                  word_reviews[word] = []
     9   6792010   64152687.0      9.4      6.4              word_reviews[word].append(recipe_id)
    10                                               
    11         1         21.0     21.0      0.0      word_reviews_count = {}
    12    126679  306783414.0   2421.7     30.8      for _, row in df.dropna(subset=['review']).iterrows():
    13    126679   44411769.0    350.6      4.5          review = row['review']
    14    126679    9895625.0     78.1      1.0          words = review.split(' ')
    15   6792010   53278554.0      7.8      5.3          for word in words:
    16   6792010  126090681.0     18.6     12.6              word_reviews_count[word] = len(word_reviews[word])
    17         1          9.0      9.0      0.0      return word_reviews_count

Чтобы оптимизировать функцию, используем один цикл для перебора строк и одновременного обновления словаря слов и связанных с ними идентификаторов просмотра. Затем мы можем один раз пройтись по словарю и подсчитать количество отзывов для каждого слова. Такой подход устраняет необходимость в нескольких циклах и вызовах функции len().

In [14]:
def get_word_reviews_count_optimized(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = set(review.split(' '))
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = set()
            word_reviews[word].add(recipe_id)
    
    word_reviews_count = {}
    for word in word_reviews:
        word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [15]:
%lprun -f get_word_reviews_count_optimized get_word_reviews_count_optimized(reviews)

    Timer unit: 1e-07 s
    Total time: 45.3822 s

Function: get_word_reviews_count_optimized at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_word_reviews_count_optimized(df):
     2         1         38.0     38.0      0.0      word_reviews = {}
     3    126679  242371884.0   1913.3     53.4      for _, row in df.dropna(subset=['review']).iterrows():
     4    126679   60488423.0    477.5     13.3          recipe_id, review = row['recipe_id'], row['review']
     5    126679   14716620.0    116.2      3.2          words = set(review.split(' '))
     6   5387307   25890758.0      4.8      5.7          for word in words:
     7   5212363   40184262.0      7.7      8.9              if word not in word_reviews:
     8    174944    2001137.0     11.4      0.4                  word_reviews[word] = set()
     9   5387307   64174646.0     11.9     14.1              word_reviews[word].add(recipe_id)
    10                                               
    11         1         20.0     20.0      0.0      word_reviews_count = {}
    12    174944    1153746.0      6.6      0.3      for word in word_reviews:
    13    174944    2840627.0     16.2      0.6          word_reviews_count[word] = len(word_reviews[word])
    14         1          3.0      3.0      0.0      return word_reviews_count

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.
